In [57]:
from ibm_quantum_widgets import CircuitComposer
from IPython.display import display, Javascript, HTML
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit.providers.aer import QasmSimulator

In [58]:
backend = QasmSimulator()

In [59]:
### VIEW: welcome
# initialize drink variable
drink = "coffee"
html = HTML("""
<h1>Welcome</h1>
<br/>
<br/>
<button class="btn btn-primary" data-q-target="circuitSelection">Get your coffee</button>
<button class="btn btn-primary" >Get more information</button>
""")
display(html)

In [60]:
# build some sample circuits
qc0 = QuantumCircuit(2, 2)

qc1 = QuantumCircuit(2, 2)
qc1.h(0)
qc1.h(1)

In [61]:
qcreg = {
    'scratch': qc0,
    'superposition': qc1,
}

In [62]:
# initialize qc variable
qc = qcreg['scratch'].copy()

In [68]:
### VIEW: circuitSelection
html = HTML("""
<h1>Where do you want to start</h1>
<br/>
<br/>
<button class="btn btn-primary" data-q-code="qc=qcreg['scratch'].copy()" data-q-target="initEditor,editor">Start from scratch</button>
<button class="btn btn-primary" data-q-code="qc=qcreg['superposition'].copy()" data-q-target="initEditor,editor">Start with superposition</button>
<button class="btn btn-primary">...</button>
<br><br>
<button class="btn btn-default" data-q-target="welcome">Back</button>
""")
display(html)

In [69]:
### VIEW: initEditor
editor = CircuitComposer(circuit=qc)

# handler for change events
def calculate(change_event):
    # there are multiple change events, only use one
    if change_event['name'] != "circuit":
        return
    # copy current circuit
    mqc = editor.circuit.copy()
    # add measurement
    mqc.measure(range(2), range(2))
    # get simulation result
    mqc_compiled = transpile(mqc, backend)
    job_sim = backend.run(mqc_compiled, shots=1024)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(mqc_compiled)
    for bit in counts:
        JsWindowState.set("editor-circuit-bit-"+bit, counts[bit]/1024)
    qasm = mqc.qasm()
    JsWindowState.set("editor-circuit-qasm", qasm)
    
# add handler to change events
editor.observe(calculate)

In [70]:
### VIEW: editor
editor

CircuitComposer(circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1455d9cd0>)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
### VIEW: editor
html = HTML("""
Coffee (00):
<div class="progress editor-circuit-probbar-00">
  <div class="progress-bar editor-circuit-probtext-00" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Cappucino (01):
<div class="progress editor-circuit-probbar-01">
  <div class="progress-bar editor-circuit-probtext-01" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Espresso (10):
<div class="progress editor-circuit-probbar-10">
  <div class="progress-bar editor-circuit-probtext-10" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Latte Macchiato (11):
<div class="progress editor-circuit-probbar-11">
  <div class="progress-bar editor-circuit-probtext-11" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>


<button class="btn btn-primary editor-circuit-max-id">Get a <span class="editor-circuit-max-name"></span></button>
<button class="btn btn-primary" data-q-open-external="editor-circuit-qasm">Transfer to IBM Quantum Composer</button>
<button class="btn btn-default" data-q-target="circuitSelection">Back</button>
""")
display(html) 

In [67]:
### VIEW: success
html = HTML("""
<h1>Success</h1>
<button class="btn btn-default" data-q-target="welcome">Back</button>
""")
display(html) 